# Test various python ordinary differential equation (ODE) solvers
## Test them on the ODE for a pendulum

In [ ]:
# For odeint, look at https://apmonitor.com/pdc/index.php/Main/SolveDifferentialEquations
import numpy as np
from scipy.integrate import odeint, RK45, solve_ivp
import matplotlib.pyplot as plt
import time

# Definitions

## The Hamiltonian pendulum
### $L = \frac{1}{2} m (l\dot\theta)^2 - mgl(1-\cos\theta)$
### $p = \partial L/\partial \dot\theta = m l^2 \dot\theta$
### $H = p\dot\theta - L = \frac{p^2}{2ml^2} - mgl\cos\theta$
### $\frac{d\theta}{dt}= \frac{\partial H}{\partial p} = \frac{p}{ml^2}$
### $\frac{d p}{dt} = -\frac{\partial H}{\partial \theta} = -mgl\sin\theta$

# Define a function that returns $d{\bf y}/dt$, where ${\bf y}=(\theta, p)$.

In [ ]:
# function that returns dy/dt
# The integrators want a function of the form f(t,y), but usually f = f(t,y, a,b,c)
# This can be handled by using a Lamda function, or by using a wrapper, or, for solve_ivp
# by using 'args=(a,b,cv)'.
#
# def model(y,t,m,g,l): 
def model_t_first(t,y,m,g,l): # odeint needs the order (y,t), setting tfirst = True reverses the order
    # For this version, set tfirst = True
    # A Hamiltonian formulation of a pendulum, H = 1/2 P^2/(ml^2) -mgl cos(theta)
    # d\theta /dt = p / (ml^2),  y[0] = \theta, y[1] = p
    # dp/dt = -mgl sin(\theta)
    theta, p = y
    dydt = [ p /(m * l**2),  \
            -m * g * l * np.sin(theta) ]
    return dydt

# Unlike solve_ivp (or most python functions), odeint needs the order (y,t)
def model_y_first(y,t,m,g,l): # odeint needs the order (y,t), setting tfirst = True reverses the order
    # For this version, tfirst should be set False
    # A Hamiltonian formulation of a pendulum, H = 1/2 P^2/(ml^2) -mgl cos(theta)
    # d\theta /dt = p / (ml^2),  y[0] = \theta, y[1] = p
    # dp/dt = -mgl sin(\theta)
    theta, p = y
    dydt = [ p /(m * l**2),  \
            -m * g * l * np.sin(theta) ]
    return dydt

# Or using a wrapper
def dydt_wrapper(t, y):
    dydt = model_t_first(t, y, m, g, l)
    return(dydt)

## Some useful functions
### The energy $E = \frac{1}{2}\frac{p^2}{ml^2} - mgl\cos\theta$
### my_mod keeps $\theta\in(-\pi,\pi)$

In [ ]:
def energy(y):
    theta = y[:,0]
    p     = y[:,1]
    kinetic_energy = 0.5 * p**2 / (m * l**2)
    potential_energy = -m * g * l * np.cos(theta)
    return(kinetic_energy, potential_energy, kinetic_energy + potential_energy)

def energy4(theta, p):
    kinetic_energy = 0.5 * p**2 / (m * l**2)
    potential_energy = -m * g * l * np.cos(theta)
    return(kinetic_energy, potential_energy, kinetic_energy + potential_energy)

def my_mod(theta): # Ensure that angles are in the range (-pi, pi) for phase space plots
    theta = np.mod(theta, 2.*np.pi)
    theta = np.where(theta > np.pi, theta - 2.*np.pi, theta)
    theta = np.where(theta < - np.pi, theta + 2.*np.pi, theta)
    return(theta)

## On the separatrix, $H = H(\pi,0) = E_{sep}= mgl$,
## while the maximum $p$ is given by $E_{sep} = \frac{p^2}{ml^2} + mgl$
## or $p_{sep} = \pm\sqrt{4ml^2E_{sep}}$

In [ ]:
# parameters
m = 100.  # grams
l = 10.   # cm
g = 980.  # cm / s^2

period = 2. * np.pi * np.sqrt(l/g)
print('Period of small oscillations= {0:2.2e} seconds'.format(period))

E_sep = m*g*l
p_sep = np.sqrt(4. * m * l**2 * E_sep)
print('Separatrix energy = {0:2.2e}'.format(E_sep))
print('Potential energy scale = mgl= {0:2.2e}'.format(m*g*l))

# Initial conditions and time span
## We will integrate for times the period of small oscillations 

In [ ]:
# time points
num_periods = 40
t_eval = np.linspace(0, num_periods * period, num = 5000) # The integrators put out y(t) at these times
t_span = [t_eval[0], t_eval[-1] ] # Begining and ending values of the time

# initial condition
y_sep = [ np.pi , 0.] # Very near the separatrix
y0 = [0.1 * np.pi , 0. ] # Near the stable fixed point
y_rotate = [0., 1.01 * p_sep] # Rotating, E just above E_separatrix
y_rotate_m = [0., -1.01 * p_sep] # Rotating, E just below E_separatrix
#
#
initial_KE, initial_PE, initial_Energy = energy4(y0[0], y0[1])
print('initial PE, KE, Energy = ', initial_PE, initial_KE, initial_Energy)

# Run the ODE solvers
## Start with the old fasioned odeint

In [ ]:
## Maximum errors (supposedly)
atol = [3.e-12, 3.e-12]
rtol = 3.e-12

# solve ODEs

# default rtol = atol = 1.5e-8, according to the online documentaion
print('rtol = {0}, atol = {1}'.format(rtol, atol))

# Call odeint using 'args=()'
y1_small, dict_ode = odeint(model_y_first, y0, t_eval, args=(m, g, l), \
                      full_output=True, rtol = rtol, atol = atol)   #Standard odeint function, f(y,t)

# Again, returning more information
y2,dict_ode2 = odeint(model_t_first, y0, t_eval, args=(m, g, l), \
                      rtol = rtol, atol = atol, full_output=True, tfirst = True) #non-odeint standard function, f(t,y)
#print('now t=', t)
print('rtol = {0}, atol = {1}'.format(rtol, atol))

#Timing stuff
start_odeint = time.time()
#
y1_sep = odeint(model_y_first, y_sep, t_eval,  args=(m, g, l), \
                       rtol = rtol, atol = atol)  
end_odeint = time.time()
run_time_odeint = end_odeint - start_odeint
print('\n', 'Odeint separatrix run time = {0:2.2e}'.format(run_time_odeint), '\n')

# Find rotating motion (rather than the librations found above)
#
y1_rotation = odeint(model_y_first, y_rotate, t_eval, args=(m, g, l), \
                       rtol = rtol, atol = atol) 

y1_rotation_m = odeint(model_y_first, y_rotate_m, t_eval, args=(m, g, l), \
                       rtol = rtol, atol = atol) 
# #
# #


# Use the new (and improved?) solve_ivp routine

In [ ]:
# Now try the new scipy ode integrator solve_ivp
#Default values are 1e-3 for rtol and 1e-6 for atol.
#
# Time solve_ivp, to compare to ODEint
#
#
start_ivp = time.time()
sol_sep = solve_ivp(fun=lambda t, y: model_t_first(t, y, m, g, l), t_span = t_span, \
                y0 = y_sep, t_eval = t_eval, rtol = rtol, atol = atol)
end_ivp = time.time()
run_time_ivp = end_ivp - start_ivp
#
#
print('\n', 'Odeint separatrix run time = {0:2.2e}'.format(run_time_odeint), '\n')

print('\n', 'ivp run time = {0:2.2e}'.format(run_time_ivp), '\n')
#
#
sol_small = solve_ivp(fun=lambda t, y: model_t_first(t, y, m, g, l), t_span = t_span, \
                y0 = y0, t_eval = t_eval, rtol = rtol, atol = atol)
sol_rotation = solve_ivp(fun=lambda t, y: model_t_first(t, y, m, g, l), t_span = t_span, \
                y0 = y_rotate, t_eval = t_eval, rtol = rtol, atol = atol)
#
# Using a wrapper for the function
#
sol_rotation_m = solve_ivp(dydt_wrapper, t_span = t_span, \
                y0 = y_rotate_m, t_eval = t_eval, rtol = rtol, atol = atol)
#
#
theta_sep = sol_sep.y[0,:]
p_sep = sol_sep.y[1,:]
theta_small = sol_small.y[0,:]
p_small = sol_small.y[1,:]
#
theta_rotation = sol_rotation.y[0,:]
p_rotation = sol_rotation.y[1,:]
theta_rotation_m = sol_rotation_m.y[0,:]
p_rotation_m = sol_rotation_m.y[1,:]
#
t4 = sol_sep.t

# restrict $\theta$ to (-$\pi$, $\pi$)

In [ ]:
#The odeint solution first
theta1_sep = my_mod(y1_sep[:,0])
theta1_small = my_mod(y1_small[:,0])
theta1_rotation = my_mod(y1_rotation[:,0])
theta1_rotation_m = my_mod(y1_rotation_m[:,0])
#
#Now the solve_ivp solution
theta_sep = my_mod(theta_sep)
theta_small = my_mod(theta_small)
theta_rotation = my_mod(theta_rotation)
theta_rotation_m = my_mod(theta_rotation_m)

In [ ]:
# plot results; theta(t) first
#
plt.plot(t_eval,theta_sep,'g--',linewidth=2,)
#
plt.xlabel('time')
plt.ylabel(r'$\theta$ (radians)')
# plt.legend()
plt.show()

In [ ]:
# Now p(t); note that we are not quite on the separatrix.
# If we were p(t) would take infinite time to go 0+ to 0+

plt.plot(t_eval, p_sep,'g--',linewidth=2,)
#
plt.xlabel('time')
plt.ylabel(r'$p\, (g\,cm^2\, s^{-1}$')
# plt.legend()
plt.show()

# Plot the phase space; the odeint results first

In [ ]:
# plot results from odeint
plt.plot(theta1_sep, y1_sep[:,1], ',')
plt.plot(theta1_small, y1_small[:,1], ',')
plt.plot(theta1_rotation, y1_rotation[:,1], ',')
plt.plot(theta1_rotation_m, y1_rotation_m[:,1], ',')

plt.xlim(-np.pi, np.pi)
plt.xlabel(r'$\theta$')
plt.ylabel('$p$')
# plt.legend()
plt.show()

# Now plot the solve_ivp results
## They are very similar to the odeint results

In [ ]:
# plot results from solve_ivp
#
plt.plot(theta_sep, p_sep, ',')
plt.plot(theta_small, p_small,',')
plt.plot(theta_rotation, p_rotation,',')
plt.plot(theta_rotation_m, p_rotation_m,',')

plt.xlim(-np.pi, np.pi)
plt.xlabel(r'$\theta$')
plt.ylabel('$p$')
# plt.legend()
plt.show()

# Plot the energy to see how accurate the integrator is

In [ ]:
# plot the energy versus time
KE, PE, Energy = energy(y1_sep)
Energy_error = (Energy - Energy[0]) / Energy[0]
plt.plot(t_eval, Energy_error, color='red')

KE4, PE4, Energy4 = energy4(sol_sep.y[0,:], sol_sep.y[1,:])
Energy_error4 = (Energy4 - Energy4[0]) / Energy4[0]
plt.plot(t_eval, Energy_error4, color = 'green')

plt.xlabel(r'$t$ (sec)')
plt.ylabel(r'$ \Delta E\,/\,E$')
# plt.legend()
plt.show()